In [1]:
import json

f = open('../Volumes/text_pcnn_att_pred.json')
pcnn_pred = json.load(f)

In [2]:
len(pcnn_pred.keys())

388920

In [28]:
f = open('../TuckER/data/UMLS-PubMed/raw_data/train.json')
dc_ = json.load(f)

f = open('../TuckER/data/UMLS-PubMed/raw_data/test_5sent.json')
tmp = json.load(f)

dc = dc_ + tmp

In [4]:
f = open('../Volumes/Aux/Downloaded/Data-Upload/UMLS+PubMed/text/rel2id.json')
rel2id = json.load(f)

In [19]:
def corpus2triples(corpus_list):
    triple_set = set()
    for item in corpus_list:
        head = item['head']['word']
        tail = item['tail']['word']
        relation = rel2id[item['relation']]
        triple = head + '\t' + str(relation) + '\t' + tail
        if triple not in triple_set:
            triple_set.add(triple)
    
    return triple_set

def corpus2queries(corpus_list):
    query_set = set()
    for item in corpus_list:
        head = item['head']['word']
        tail = item['tail']['word']
        query = head + '\t' + tail
        if query not in query_set:
            query_set.add(query)
    
    return query_set


In [29]:
test_triple_set = corpus2triples(tmp)
test_query_set = corpus2queries(tmp)
all_triple_set = corpus2triples(dc)
train_triple_set = corpus2triples(dc_)
len(test_triple_set), len(all_triple_set)

(388954, 419140)

In [38]:
count = 0
for item in dc:
    if item['relation'] != "NA":
        count += 1
print(count)

4735238


In [39]:
from tqdm import tqdm

rel_list = [
    'gene_associated_with_disease',
    'disease_has_associated_gene',
    'gene_mapped_to_disease',
    'disease_mapped_to_gene',
    'may_be_treated_by',
    'may_treat',
    'may_be_prevented_by',
    'may_prevent',
]

id_list = [str(rel2id[x]) for x in rel_list]


def filter(entity_pair, score_list, res):
    head, tail = entity_pair.split("#")
    for item in score_list:
        if float(item['score']) > 0.5:
            triple = head + '\t' + str(item['relation']) + '\t' + tail
            res.add(triple)
            break
        else:
            break


added_edges = set()
for key in tqdm(pcnn_pred.keys()):
    filter(key, pcnn_pred[key], added_edges)

100%|██████████| 388920/388920 [00:00<00:00, 516751.21it/s]


In [40]:
id_cnt = {}
for id in id_list:
    id_cnt[id] = 0

for edge in added_edges:
    relation_id = edge.split('\t')[1]
    id_cnt[relation_id] += 1

print('added distribution: ', id_cnt)

for id in id_list:
    id_cnt[id] = 0

for edge in train_triple_set:
    relation_id = edge.split('\t')[1]
    if relation_id in id_list:
        id_cnt[relation_id] += 1

print('train distributoon: ', id_cnt)


added distribution:  {'132': 0, '165': 0, '184': 0, '177': 0, '32': 157024, '29': 35463, '33': 0, '103': 0}
train distributoon:  {'132': 31, '165': 36, '184': 19, '177': 27, '32': 801, '29': 781, '33': 138, '103': 128}


In [42]:
added_edges_new = set()
for item in added_edges:
    head, tail = item.split('\t')[0], item.split('\t')[2]
    query = head + '\t' + tail
    if query in test_query_set:
        added_edges_new.add(item)

len(added_edges_new), len(test_query_set)

(192487, 388954)

In [36]:
len(added_edges), len(added_edges.intersection(train_triple_set))

(18854, 0)

In [53]:
added_edges_filt = set()

for item in added_edges:
    head, relation, tail = item.split('\t')
    if relation not in id_list:
        continue
    else:
        added_edges_filt.add(item)
len(added_edges_filt)

0

In [55]:
rel_set_ = set()

for item in added_edges:
    rel_set_.add(item.split('\t')[1])
rel_set_

{'14', '6'}

In [47]:
f = open('../TuckER/data/UMLS-PubMed/train_100.txt')

train_joint = ""
kg_train = f.readlines()

for item in kg_train:
    train_joint += item

for item in added_edges:
    head, rel_id, tail = item.split('\t')
    if rel_id == '32':
        relation = 'may_be_treated_by'
    if rel_id == '29':
        relation = 'may_treat'
    triple = head + '\t' + relation + '\t' + tail + '\n'
    train_joint += triple

train_j = open("./train.txt", 'w', encoding='utf-8')
print(train_joint, file=train_j)


{'C0903890\t29\tC0332197',
 'C0007299\t32\tC0678723',
 'C1701061\t32\tC0020861',
 'C0037903\t32\tC0005847',
 'C0054668\t32\tC0011065',
 'C1185741\t32\tC1826449',
 'C0018606\t32\tC1254351',
 'C0039441\t32\tC0020364',
 'C0012472\t32\tC0026764',
 'C0061863\t32\tC0544452',
 'C0065042\t32\tC0017243',
 'C0127615\t32\tC0205447',
 'C0206672\t32\tC0010709',
 'C0027962\t32\tC0027651',
 'C0020336\t32\tC0439228',
 'C0041341\t32\tC0523744',
 'C0597357\t32\tC0010633',
 'C0018767\t32\tC0018681',
 'C0008294\t29\tC0025859',
 'C1511737\t32\tC0040300',
 'C0599880\t32\tC0376690',
 'C0991560\t29\tC0032487',
 'C0206492\t32\tC0032105',
 'C0021368\t32\tC0175701',
 'C0008013\t32\tC0004096',
 'C0398739\t29\tC0079189',
 'C0040715\t32\tC0040845',
 'C0168273\t32\tC0332281',
 'C0220769\t32\tC0030705',
 'C0002066\t32\tC0086582',
 'C2931187\t32\tC1845055',
 'C1312139\t32\tC0021223',
 'C0032032\t32\tC0444505',
 'C0037135\t32\tC0242724',
 'C0073085\t29\tC0596901',
 'C0458952\t32\tC0015385',
 'C0076681\t32\tC0439801',
 